In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from arboreto.algo import grnboost2
from dask.distributed import Client, LocalCluster
import scipy.sparse
import utils
import rds2py
import os.path
import datetime
import dask.array

n_workers=20
n_iter=100

mcg_dat_path = "/space/scratch/amorin/aggregate_microglia/microglia_dat_list.RDS"
mcg_meta_path = "/space/scratch/amorin/aggregate_microglia/microglia_metadata_dedup.tsv"
pc_hg_path = "/home/amorin/Data/Metadata/refseq_select_hg38.tsv"
pc_mm_path = "/home/amorin/Data/Metadata/refseq_select_mm10.tsv"
tfs_hg_path = "/home/amorin/Data/Metadata/TFs_human.tsv"
tfs_mm_path = "/home/amorin/Data/Metadata/TFs_mouse.tsv"

out_dir = "/space/scratch/amorin/aggregate_microglia/GRNBoost2"
Path(out_dir).mkdir(exist_ok=True)

In [ ]:
local_cluster = LocalCluster(n_workers=n_workers, threads_per_worker=1)
custom_client = Client(local_cluster)

In [2]:
mcg_meta = pd.read_table(mcg_meta_path)
pc_hg = pd.read_table(pc_hg_path)
pc_mm = pd.read_table(pc_mm_path)
tfs_hg = pd.read_table(tfs_hg_path)
tfs_mm = pd.read_table(tfs_mm_path)

In [9]:
ids_hg = mcg_meta[mcg_meta.Species == "Human"]["ID"].tolist()
ids_mm = mcg_meta[mcg_meta.Species == "Mouse"]["ID"].tolist()

In [12]:
ids_mm = ids_mm.reverse()

In [ ]:
print(ids_mm)

In [ ]:
def iter_grnboost2(mat, genes, tfs, n_iter, out_dir, client):
    
    for i in range(n_iter):
        
        network = grnboost2(expression_data=mat, 
                            tf_names=tfs,
                            gene_names=genes,
                            seed=i,
                            client_or_address=client)
        
        file_path = Path(out_dir, f"GRNBoost2_TFonly_iter{i}.tsv")
        network.to_csv(file_path, sep="\t", header=False, index=False)
        
    return None

In [ ]:
def iter_grnboost2_over_list(dat_list, 
                             ids, 
                             gene_df, 
                             tf_df, 
                             n_iter, 
                             out_root, 
                             client):
    
    for id in ids:
        
        print(f"{id}", datetime.datetime.now())
        out_dir = Path(out_root, id)
        out_dir.mkdir(exist_ok=True)

        mat = dat_list[id]["Mat"]
        keep_ix = mat.getnnz(axis=1) >= 20  # keep genes measured in 20+ cells
        mat_sub = mat[keep_ix, ].T  # to cells by genes
        genes_sub = gene_df.iloc[keep_ix]["Symbol"].tolist()
        tfs_sub = list(set(genes_sub).intersection(tf_df["Symbol"]))

        iter_grnboost2(mat_sub, genes_sub, tfs_sub, n_iter, out_dir, client)

    return None

In [ ]:
mcg_dat = rds2py.read_rds(mcg_dat_path)

In [ ]:
mat = mcg_dat["GSE180928"]["Mat"]  # CSC mat
ix = mat.getnnz(axis=1) >= 20

In [ ]:
# mat = pd.DataFrame.sparse.from_spmatrix(mat)  # So slow
# ix = mat_df.index[(mat_df > 0).sum(axis=1) >= 20]  

In [ ]:
# mat = mat.toarray()  # np array fast but not quite as fast as sparse
# ix = np.sum(mat > 0, axis=1) >= 20

In [ ]:
mat_sub = mat[ix, ].T
# mat_sub = dask.array.from_array(mat_sub)  # Don't think this helps
genes_sub = pc_hg.iloc[ix]["Symbol"].tolist()
tfs_sub = list(set(genes_sub).intersection(tfs_hg["Symbol"]))

In [ ]:
network = grnboost2(expression_data=mat_sub, 
                    tf_names=tfs_sub,
                    gene_names=genes_sub,
                    seed=5,
                    client_or_address=custom_client)

In [ ]:
iter_grnboost2_over_list(dat_list=mcg_dat, 
                         ids = [ids_hg[0]], 
                         gene_df=pc_hg, 
                         tf_df=tfs_hg, 
                         n_iter=1,
                         out_root=out_dir,
                         client=custom_client)

In [ ]:
network.head()

In [27]:
custom_client.close()
local_cluster.close()